In [67]:
import logging
import re
import sys
from bs4 import BeautifulSoup
from queue import Queue
from urllib import parse, request
from urllib.parse import urlparse
import heapq
from urllib.robotparser import RobotFileParser
from selenium import webdriver 

In [130]:
site = "https://ocw.mit.edu/search/?d=Electrical%20Engineering%20and%20Computer%20Science"
page = requests.get(site)
#print(page.text)
soup = BeautifulSoup(page.content, "html.parser")

##### OCW MIT

Starting URL: https://ocw.mit.edu/search/?d=Electrical%20Engineering%20and%20Computer%20Science&q=computer%20science 

Courses Format:https://ocw.mit.edu/courses

Course1: https://ocw.mit.edu/courses/6-001-structure-and-interpretation-of-computer-programs-spring-2005/
Course2: https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-fall-2010/

Course Format: https://ocw.mit.edu/courses/[course # , title, date created]/
- EECS courses seem to be 6.###A-Z

Syllabus: https://ocw.mit.edu/courses/6-001-structure-and-interpretation-of-computer-programs-spring-2005/pages/syllabus/

Format: courseurl + "pages/syllabus/"

Example of graduate course: https://ocw.mit.edu/courses/mas-865j-quantum-information-science-spring-2006/
- mas for masters? 


Title

Course Description

Syllabus url


In [141]:
# main 
links = get_links(site)
writelines('links.txt', links)

print(links)

nonlocal_links = get_nonlocal_links(site)
writelines('nonlocal.txt', nonlocal_links)

visited, extracted = crawl(site)
#writelines('visited.txt', visited)
#writelines('extracted.txt', extracted)
#print(nonlocal_links)
print("visited", visited)
print("-------------------")
print(extracted)

[('https://ocw.mit.edu/', ''), ('https://giving.mit.edu/give/to/ocw/?utm_source=ocw&utm_medium=homepage_banner&utm_campaign=nextgen_home', 'Give Now'), ('https://ocw.mit.edu/about', 'About OCW'), ('https://mitocw.zendesk.com/hc/en-us', 'Help & Faqs'), ('https://ocw.mit.edu/contact', 'Contact Us'), ('https://ocw.mit.edu/', ''), ('https://giving.mit.edu/give/to/ocw/?utm_source=ocw&utm_medium=homepage_banner&utm_campaign=nextgen_home', ''), ('https://ocw.mit.edu/about', 'about ocw'), ('https://mitocw.zendesk.com/hc/en-us', 'help & faqs'), ('https://ocw.mit.edu/contact', 'contact us'), ('https://accessibility.mit.edu', 'Accessibility'), ('https://creativecommons.org/licenses/by-nc-sa/4.0/', 'Creative Commons License'), ('https://ocw.mit.edu/pages/privacy-and-terms-of-use/', 'Terms and Conditions')]
name 'visitlog' is not defined https://ocw.mit.edu/search/?d=Electrical%20Engineering%20and%20Computer%20Science
visited ['https://ocw.mit.edu/search/?d=Electrical%20Engineering%20and%20Computer

In [132]:
def parse_links(root, html):
    soup = BeautifulSoup(html, 'html.parser')
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            text = link.string
            if not text:
                text = ''
            text = re.sub('\s+', ' ', text).strip()
            yield (parse.urljoin(root, link.get('href')), text) #HERE

In [138]:
def is_self_referencing_link(url, link): #check if this is right
    end_of_link = url.rsplit('/', 1)[-1]
    return end_of_link in link

In [133]:
def get_nonlocal_links(url):
    '''Get a list of links on the page specificed by the url,
    but only keep non-local links and non self-references.
    Return a list of (link, title) pairs, just like get_links()'''

    # TODO: implement
    domain = ""
    links = get_links(url)
    filtered = []

    for link, title in links: 
        if urlparse(link).netloc != domain:
            if not is_self_referencing_link(url, link):
                filtered.append((link, title))
    return filtered

In [151]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_links(url):
    driver = webdriver.Chrome()  # using Selenium
    driver.get(url)  # link to scrape

    res = driver.page_source
    driver.quit()  # selenium

    soup = BeautifulSoup(res, 'html.parser')
    links = []
    
    #, class_='lr-row course-title'

    # Find all divs with class "lr-row course-title" and then find the <a> tags within them
    for div in soup.find_all('div'):
        print("DIV", div)
        for a in div.find_all('a', href=True):
            print("A", a)
            href = a.get('href')
            if href:
                full_url = urljoin(url, href)
                links.append(full_url)

    return links



# Example usage
url = 'https://ocw.mit.edu/search/?d=Electrical%20Engineering%20and%20Computer%20Science'
links = get_links(url)
print(links)


DIV <div>
<div class="search-wrapper">
<div class="position-relative bg-black medium-and-below-only" id="mobile-header">
<nav class="navbar navbar-expand-lg navbar-dark bg-black d-flex">
<button aria-controls="navbarSupportedContent" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarSupportedContent" data-toggle="collapse" type="button">
<i class="material-icons display-4 text-white align-bottom">menu</i>
</button>
<div class="mx-auto">
<a href="/">
<img alt="MIT OpenCourseWare" src="/static_shared/images/ocw_logo_white.cabdc9a745b03db3dad4.svg" width="250"/>
</a>
</div>
<div class="collapse navbar-collapse" id="navbarSupportedContent">
<ul class="navbar-nav mr-auto pl-2 pt-2">
<li class="nav-item">
</li>
<li class="nav-item">
<a class="nav-link" href="https://giving.mit.edu/give/to/ocw/?utm_source=ocw&amp;utm_medium=homepage_banner&amp;utm_campaign=nextgen_home">Give Now</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/about">Abo

In [134]:
def get_links(url):
    driver = webdriver.Chrome() # using Selenium 
    driver.get(url) #link to scrape 
    res = driver.page_source

    #res = request.urlopen(url)
    driver.quit() #selenium

    return list(parse_links(url, res))

In [135]:
def writelines(filename, data):
    with open(filename, 'w') as fout:
        for d in data:
            print(d, file=fout)

In [136]:
def crawl(root, wanted_content=[], within_domain=True):
    '''Crawl the url specified by `root`.
    `wanted_content` is a list of content types to crawl
    `within_domain` specifies whether the crawler should limit itself to the domain of `root`
    '''
    # TODO: implement

    queue = Queue()
    queue.put(root)

    visited = []
    extracted = []

    while not queue.empty(): #and len(visited) < 200
        url = queue.get()
        try:
            req = request.urlopen(url)
            html = req.read()

            if url not in visited:
                visited.append(url)
                visitlog.debug(url)

                #test uncomment    
                for ex in extract_information(url, html):
                    extracted.append(ex)
                    extractlog.debug(ex)

                for link, title in parse_links(url, html):
                    if not is_self_referencing_link(url, link): 
                        if within_domain and is_within_domain(url, link):
                            if len(wanted_content) == 0: 
                                queue.put(link)
                            elif len(wanted_content) > 0 and req.headers['Content-Type'] in wanted_content:
                                queue.put(link)                

                    soup = BeautifulSoup(html, 'html.parser')
                    div = soup.find('div', class_='mt-2 lead-sm html-data')
                    if div:
                        text = div.text.strip()
                        extracted.append((url, 'HTML_DATA', text))
                    for link, title in parse_links(url, html):
                        if not is_self_referencing_link(url, link): 
                            if within_domain and is_within_domain(url, link):
                                if len(wanted_content) == 0: 
                                    queue.put(link)
                                elif len(wanted_content) > 0 and req.headers['Content-Type'] in wanted_content:
                                    queue.put(link)                


        except Exception as e:
            print(e, url)

    return visited, extracted